In [1]:
#| label: libraries
#| include: false
import pandas as pd 
import numpy as np
import plotly.express as px
import sqlite3 as sql

## Elevator pitch

__In the following data analysis I explore the statistics behind Baseball. Baseball is a numbers game, so data exploration like the one below provides numerous insights into the performance of players and teams over time, and why. I was able to discover changes in salary overtime, explore the batting average statistic, and compared the historical skill of two famous MLB teams.__


In [2]:
#| label: project-data
#| code-summary: Read and format project data

# Data connection
sqlite_file = 'lahmansbaseballdb.sqlite'
con = sql.connect(sqlite_file)

q = '''
    SELECT * 
    FROM sqlite_master 
    WHERE type='table'
    '''
table = pd.read_sql_query(q,con)
table.filter(['name'])

,name
0,allstarfull
1,appearances
2,awardsmanagers
3,awardsplayers
4,awardssharemanagers
5,awardsshareplayers
6,batting
7,battingpost
8,collegeplaying
9,divisions


## Write an SQL query to create a new dataframe about baseball players who attended BYU-Idaho

__The new table should contain five columns: playerID, schoolID, salary, and the yearID/teamID associated with each salary. Order the table by salary (highest to lowest) and print out the table in your report.__

The formatted table below shows the 2 professional players fround from BYUI. The statistics display the years they played, for which team, and their salary for the given year, in descending order by salary. This gives us insights on the annual salary made by the students over the years. Naturally, the longer a student plays the more they seem to make. We also notice that the player with id 'lindsma01' generally made more than 'stephga01'. This is likely due to the fact that he played about 10 years later, thus inflation and general industry/team growth would have increased this amount naturally. 


In [3]:
#| label: Q1
#| code-summary: Format data
# Creating table
player_q = '''
    SELECT *
    FROM collegeplaying
    WHERE schoolID == "idbyuid"
    '''

results = pd.read_sql_query(player_q, con)
byuiPlayers = results['playerID']

salary_q = '''
  SELECT *
  FROM salaries
  WHERE playerID == "stephga01" or playerID == "lindsma01"
  ORDER BY salary DESC
  '''
salaries = pd.read_sql_query(salary_q, con)
salaries['schoolID'] = "idbyuid"
salary_df = salaries.drop(['lgID', 'ID', 'team_ID'], axis=1)

In [4]:
#| label: Q1-table
#| code-summary: Table to show BYUI players
#| tbl-cap: BYUI Players Table
#| tbl-cap-location: top
# table
salary_df

,yearID,teamID,playerID,salary,schoolID
0,2014,CHA,lindsma01,4000000.0,idbyuid
1,2012,BAL,lindsma01,3600000.0,idbyuid
2,2011,COL,lindsma01,2800000.0,idbyuid
3,2013,CHA,lindsma01,2300000.0,idbyuid
4,2010,HOU,lindsma01,1625000.0,idbyuid
5,2001,SLN,stephga01,1025000.0,idbyuid
6,2002,SLN,stephga01,900000.0,idbyuid
7,2003,SLN,stephga01,800000.0,idbyuid
8,2000,SLN,stephga01,550000.0,idbyuid
9,2009,FLO,lindsma01,410000.0,idbyuid


## Calculate Batting Average

The following three tables show batting percentage statistics in different forms and gives us insights into historical batting accuracy. In order to create these tables I needed to group the stats by playerID and create a batting percentage column from the 'at bats' and 'hits' columns.


In [5]:
#| label: Q2
#| code-summary: Format data
# Table creation
batting_q = '''
  SELECT * 
  FROM batting
  WHERE ab >= 1
  '''
batting_df = pd.read_sql_query(batting_q, con)
batting = batting_df[['playerID', 'yearID', 'AB', 'H']]
batting['bat%'] = 100*round(batting['H']/batting['AB'], 4)
batting = batting.sort_values('bat%', ascending=False)

C:\Users\alext\AppData\Local\Temp\ipykernel_4868\2409195162.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batting['bat%'] = 100*round(batting['H']/batting['AB'], 4)


This first table simply shows the top 5 batting percentages ever recorded. It makes sense to see that all 5 are people who only had 1 or 2 at bats, and thus easily managed a 100% batting percentage. 


In [6]:
#| label: Q2-table A
#| code-summary: Highest bat% table
#| tbl-cap: Highest bat% table
#| tbl-cap-location: top
# Batting % Table
display(batting.head(5).sort_values('playerID'))

,playerID,yearID,AB,H,bat%
41947,acklefr01,1964,1,1,100.0
87881,hollade01,2017,1,1,100.0
68685,magnami01,1998,2,2,100.0
39731,rushbo01,1960,1,1,100.0
74203,valvejo01,2003,1,1,100.0


The next table becomes somewhat more realistic, as it filters any player who had less than 10 at bats for the given year. Still, these players only had a handful of at bats, and thus easily achieved a very high hitting percentage.


In [7]:
#| label: Q2-table B
#| code-summary: Best batters  after 10
#| tbl-cap: Best batter with 10+ AB
#| tbl-cap-location: top
# Batting % after 10 Table
batting_10 = batting.drop(batting[batting['AB']<10].index)
display(batting_10.head(5))

,playerID,yearID,AB,H,bat%
50119,nymanny01,1974,14,9,64.29
83515,carsoma01,2013,11,7,63.64
12012,altizda01,1910,10,6,60.00
50649,johnsde01,1975,10,6,60.00
32819,silvech01,1948,14,8,57.14


The final table shown is by far the most fascinating. This summed every player's at bats in order to find the highest career batting percentage in the MLB. It only checks players with at least 100 career at bats, and results in Ty Cobb having the highest all time batting average at 36.6%.


In [8]:
#| label: Q2-table C
#| code-summary: Best career batters
#| tbl-cap: Best batter career
#| tbl-cap-location: top
# Batting % after career Table
batting_career = batting.groupby(['playerID'])[['AB', 'H']].sum().reset_index()
batting_career['bat%'] = 100*round(batting_career['H']/batting_career['AB'], 4)
batting_career = batting_career.drop(batting_career[batting_career['AB']<100].index).sort_values('bat%', ascending=False)
display(batting_career.head(5))

,playerID,AB,H,bat%
2918,cobbty01,11436,4189,36.63
754,barnero01,2391,860,35.97
7292,hornsro01,8173,2930,35.85
7609,jacksjo01,4981,1772,35.58
10616,meyerle01,1443,513,35.55


## Pick any two baseball teams and compare them using a metric of your choice

__Write an SQL query to get the data you need, then make a graph using Plotly Express to visualize the comparison. What do you learn?__

The following line chart compares the number of home runs achieved each year by the Boston Red Sox and New York Yankees. There are several fascinating insights to be found here, firstly I noticed that there is a positive trendline for both teams. This indicates that as the sport has matured, and players' skill increased overtime, it seems more and more home runs have been occuring over the last century. In terms of actual team comparison, both are relatively even, but New York seems to generally have the upper hand in home runs. What I find most interesting, is the very large gap in the 20s and 30s. Note taht the year of this large spike (1920), happens to be the year that the famous 'Babe Ruth' signed with the New York Yankees. Furthermore, we see the number of home runs drastically drop closer to the level of Boston in 1935. Babe Ruth left the Yankees in 1934. It is incredible to see one man seemingly make such a large impact on the performance of a team. 


In [9]:
#| label: Q3
#| code-summary: Format data
# Home Run comparison
team_q = '''
  SELECT *
  FROM teams
  WHERE teamID == "NYA" or teamID == "BOS"
  '''
team_df = pd.read_sql_query(team_q, con)[['teamID', 'HR', 'yearID']]

In [10]:
#| label: Q3-chart
#| code-summary: Team comparison plot
#| fig-cap: Team Comparison
#| fig-align: center
# Comparing Boston and New York
px.line(team_df, x='yearID', y='HR', title='Home runs over time', color='teamID')